In [6]:
import requests
import pandas as pd
import redshift_connector
from dotenv import dotenv_values
import psycopg2
import os
    
env_vars = dotenv_values('.env')

In [7]:
function = 'TIME_SERIES_WEEKLY'
# Big Five Tech: Google, Amazon, Meta, Apple, and Microsoft (GAMAM)
symbols = ['GOOG', 'AMZN', 'METV', 'AAPL', 'MSFT']
api_key = 'WNHSBPLUX5B8HNMZ'

df = pd.DataFrame()

for symbol in symbols:
    url = f'https://www.alphavantage.co/query?function={function}&symbol={symbol}&apikey={api_key}'
    r = requests.get(url)
    data = r.json()

    symbol_df = pd.DataFrame(data['Weekly Time Series'])
    symbol_df = symbol_df.T
    symbol_df.reset_index(inplace=True)
    symbol_df.rename(columns={'index':'week_from', '1. open':'open', '2. high':'high', '3. low': 'low', 
                       '4. close':'close', '5. volume':'volume'}, inplace=True)
    
    symbol_df['symbol'] = data['Meta Data']['2. Symbol']

    symbol_df['open'] = pd.to_numeric(symbol_df['open'])
    symbol_df['close'] = pd.to_numeric(symbol_df['close'])
    symbol_df['avg'] = (symbol_df['open'] + symbol_df['close'])/2
    # symbol_df['avg'] = symbol_df[['close', 'open']].mean(axis=1)
    
    df = pd.concat([df, symbol_df])

In [8]:
df.head()

,week_from,open,high,low,close,volume,symbol,avg
0,2023-06-28,121.4656,122.7200,116.9100,121.08,70117748,GOOG,121.2728
1,2023-06-23,123.5350,125.1750,119.6000,123.02,95665041,GOOG,123.2775
2,2023-06-16,123.3950,126.7000,122.1600,124.06,147453173,GOOG,123.7275
3,2023-06-09,124.6100,129.5500,122.0100,122.87,121430664,GOOG,123.7400
4,2023-06-02,126.2900,126.7450,122.8900,125.23,113094630,GOOG,125.7600


In [9]:
# Connect to Redshift using psycopg2
conn = psycopg2.connect(
    host=env_vars['HOST'],
    port=int(env_vars['PORT']),
    database=env_vars['DATABASE'],
    user=env_vars['USER'],
    password=env_vars['PASSWORD']
 )

cursor = conn.cursor()

# # Eliminate the table if exists
# cleaner_query = '''DROP TABLE IF EXISTS laureanoengulian_coderhouse.big_five_weekly;'''
# cursor.execute(cleaner_query)
# conn.commit()

# # Create table if not exists
# create_table_query = '''CREATE TABLE IF NOT EXISTS laureanoengulian_coderhouse.big_five_weekly(
#                         "week_from" date not null,
#                         "open" decimal(38, 4) not null,
#                         "high" decimal(38, 4) not null,
#                         "low" decimal(38, 4) not null,
#                         "close" decimal(38, 4) not null,
#                         "volume" bigint not null,
#                         "symbol" varchar(15) not null,
#                         "avg" decimal(38,4) not null);
#                      '''

# cursor.execute(create_table_query)
# conn.commit()

In [10]:
def copy_from_file(conn, df, table):
    """
    Here we are going save the dataframe on disk as 
    a csv file, load the csv file  
    and use copy_from() to copy it to the table
    """
    # Save the dataframe to disk
    tmp_df = "./tmp_dataframe.csv"
    df.to_csv(tmp_df, index=False, header=False)
    f = open(tmp_df, 'r')
    cursor = conn.cursor()
    try:
        cursor.copy_from(f, table, sep=",")
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        os.remove(tmp_df)
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("copy_from_file() done")
    cursor.close()
    os.remove(tmp_df)

In [11]:
# copy_from_file(conn, df, 'big_five_weekly') # copy the dataframe to SQL
# conn.close() # close the connection